In [1]:
!nvidia-smi

Tue Aug 23 09:10:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!gdown --id '1MOC3LBT2Wo8uiFQEKd-y4IQImImoiWg6' --output train.csv
!gdown --id '1aOqUEZceg2XQmC58Ye-wLR_fdtzV_PGt' --output test.csv
!gdown --id '1Vt5g2oaeyK6BOjDEh3r8kVhxYjnJphqs' --output Gender.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1MOC3LBT2Wo8uiFQEKd-y4IQImImoiWg6
To: /content/train.csv
100% 61.2k/61.2k [00:00<00:00, 58.6MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1aOqUEZceg2XQmC58Ye-wLR_fdtzV_PGt
To: /content/test.csv
100% 28.6k/28.6k [00:00<00:00, 41.2MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [3]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np 
import pandas as pd
import random 
from tqdm.auto import tqdm 
from pathlib import Path

In [4]:
train_raw_data = pd.read_csv('train.csv')
mosts = train_raw_data.mode().iloc[0]
train_raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
mosts

PassengerId                      1
Survived                       0.0
Pclass                         3.0
Name           Abbing, Mr. Anthony
Sex                           male
Age                           24.0
SibSp                          0.0
Parch                          0.0
Ticket                        1601
Fare                          8.05
Cabin                      B96 B98
Embarked                         S
Name: 0, dtype: object

In [6]:
train_raw_data = train_raw_data.drop(['Name', 'Cabin', 'Ticket'], axis=1)
train_raw_data['Age'] = train_raw_data['Age'].fillna(train_raw_data['Age'].mean())
train_raw_data['Embarked'] = train_raw_data['Embarked'].fillna(mosts['Embarked'])
train_raw_data['LogFare'] = np.log1p(train_raw_data['Fare'])
train_raw_data['LogAge'] =  np.log1p(train_raw_data['Age'])
train_raw_data = pd.get_dummies(train_raw_data,columns=["Sex","Pclass","Embarked"])
train_raw_data['Family'] = train_raw_data['SibSp'] + train_raw_data['Parch']
train_raw_data = train_raw_data.drop(['Parch','PassengerId','SibSp'],axis=1)
train_raw_data = train_raw_data.select_dtypes(np.number)
survived_ans = torch.tensor(train_raw_data['Survived'].values,dtype=torch.long)
train_raw_data = train_raw_data.drop(['Survived'],axis = 1)

In [7]:
titles = train_raw_data.columns
correlation = np.where(train_raw_data.corr("kendall").iloc[1] > 0.)
correlation

(array([ 0,  1,  2,  3,  4,  6,  7,  9, 12]),)

In [8]:
titles = list(titles[correlation])
titles

['Age',
 'Fare',
 'LogFare',
 'LogAge',
 'Sex_female',
 'Pclass_1',
 'Pclass_2',
 'Embarked_C',
 'Family']

In [9]:
tensor_data = torch.tensor(train_raw_data[titles].values,dtype = torch.float)
print(survived_ans, tensor_data)

tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,

In [10]:
def seeds_setup(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.dererministic = True

seeds_setup(909)

In [11]:
class TrainSet(Dataset):
    def __init__(self,data,label,mode):
        self.data = data
        self.label = label

    def __getitem__(self,idx):
            return self.data[idx], self.label[idx]
        
    def __len__(self):
            return len(self.data)

In [12]:
train_dataset = TrainSet(tensor_data, survived_ans, mode='train')

In [13]:
BATCH_SIZE = 16
train_loaded  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [14]:
len(tensor_data[0])

9

In [15]:
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel,self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(9,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p=0.8),
            nn.Linear(512,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.8),
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p=0.8),
            nn.Linear(32,2),
        )
    def forward(self,x):
        output = self.fc_layer(x)
        return output

In [16]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
early_stop = 50

EPOCHS = 500
LR = 0.0003
loss_fn = nn.CrossEntropyLoss()
os.makedirs('model', exist_ok=True)

model = LinearModel().to(device)
optimizer = torch.optim.Adam(model.parameters() , lr = LR)
model.train()

early_count = 0
max_acc = 0

model_path  = f'model/titanic.ckpt'

for epoch in range(EPOCHS):
    model.train()

    total = 0
    tp_fp = 0
    train_losses = []
    for batch in train_loaded:
        x, y = batch
        x_train = x.to(device)
        y_train = y.to(device)

        output = model(x_train)
        loss = loss_fn(output,y_train)
        preds = torch.argmax(output,dim=-1)
        tp_fp += (preds==y_train).sum().item()
        total += y_train.shape[0]
        loss.backward()
        
        train_losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad()

    train_acc = tp_fp / total
    train_loss = sum(train_losses) / len(train_losses)

    print(f'[Epochs = {epoch}] Train Accuracy = {train_acc:.5f}, Train Loss = {train_loss:.5f}')

    if train_acc > max_acc:
        max_acc = train_acc
        torch.save(model.state_dict(), model_path)
        print(f'Save model with accuracy = {train_acc:.5f}')
        stop_count = 0
    else:
        stop_count +=1
        if early_count > early_stop:
            print('Early Stop!!')
            break

[Epochs = 0] Train Accuracy = 0.57955, Train Loss = 0.74908
Save model with accuracy = 0.57955
[Epochs = 1] Train Accuracy = 0.56364, Train Loss = 0.74916
[Epochs = 2] Train Accuracy = 0.58182, Train Loss = 0.72048
Save model with accuracy = 0.58182
[Epochs = 3] Train Accuracy = 0.58750, Train Loss = 0.70261
Save model with accuracy = 0.58750
[Epochs = 4] Train Accuracy = 0.60795, Train Loss = 0.69952
Save model with accuracy = 0.60795
[Epochs = 5] Train Accuracy = 0.60000, Train Loss = 0.68481
[Epochs = 6] Train Accuracy = 0.61023, Train Loss = 0.69705
Save model with accuracy = 0.61023
[Epochs = 7] Train Accuracy = 0.63750, Train Loss = 0.67083
Save model with accuracy = 0.63750
[Epochs = 8] Train Accuracy = 0.61705, Train Loss = 0.68075
[Epochs = 9] Train Accuracy = 0.61705, Train Loss = 0.68277
[Epochs = 10] Train Accuracy = 0.62159, Train Loss = 0.67207
[Epochs = 11] Train Accuracy = 0.61023, Train Loss = 0.67015
[Epochs = 12] Train Accuracy = 0.63068, Train Loss = 0.66503
[Epochs

In [17]:
class TestSet(Dataset):
    def __init__(self,data,label,mode):
        self.data = data
        self.label = label
              
    def __getitem__(self,idx):
            return self.data[idx]
    def __len__(self):
            return len(self.data)

In [18]:
test_data = pd.read_csv('test.csv')

test_data = test_data.drop('Cabin', axis=1)
test_data['LogFare'] = np.log1p(test_data['Fare'])
test_data['LogAge'] =  np.log1p(test_data['Age'])
test_data = pd.get_dummies(test_data,columns=["Sex","Pclass","Embarked"])
test_data['Family'] = test_data['SibSp'] + test_data['Parch']
test_data = test_data.drop(['Parch','SibSp'],axis=1)
test_data = test_data.select_dtypes(np.number)
test_ID = test_data['PassengerId'].values
test_data = test_data.drop(['PassengerId'],axis=1)
test_tensor = torch.tensor(test_data[titles].values,dtype = torch.float)

In [19]:
test_dataset = TestSet(test_tensor, label=None,mode =  'test')
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE) 
model_test = LinearModel().to(device)
model_test.load_state_dict(torch.load(model_path))
model_test.eval()
models = [model_test]
pred_result = []
for batch in test_loader:
    datas = batch
    datas = datas.to(device)
    for j, model in enumerate(models):
        if j == 0: 
          output = model(datas)
        else: 
          output += model(datas)
    preds = torch.argmax(output, dim=-1)
    pred_result +=list(preds)

with open('submission.csv','w') as f:
    f.write('PassengerId,Survived\n')
    for idx,pred in zip(test_ID , pred_result):
        f.write(f'{idx},{pred}\n')